In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from os import environ
import datetime as dt
import pandas as pd

In [ ]:
environ["GOOGLE_API_KEY"] = ""

data = CSVLoader("data.csv", encoding="ISO-8859-1").load()
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0, max_tokens=None, timeout=None, max_retries=5)

prompt = ChatPromptTemplate.from_template("""
    # You're a young programmer and wants to analyze your historical job data.
    # This data set was scrappd from your linkedin profile and your mission is create new column with the following categories:
    
    # With the Position column, you can get the Role and Seniority categories.
    # With the Location column, you can get the Model and Region categories.
    # With the Last Status column, you can get the Status category.
    # With the Last Status column you also can get the Date category, for that use (Years, Months, Days) as the format.

    # The Role category must be one of the following:
    - RPA, here is anything related to automation
    - DevOps
    - Data
    - SRE

    # The Seniority category must be one of the following:
    - Intern
    - Junior
    - Mid
    - Senior
    - Lead
    - Manager

    # The Region category must be one of the following:
    - North America
    - South America
    - Europe
    - Asia
    - Africa
    - Oceania

    # The Model category must be one of the following:
    - Remote
    - Onsite
    - Hybrid

    # The Status category must be one of the following:
    - Applied
    - Viewed
    - Downloaded
    - Untracked, if was applied in the company site

    # The data set is as follows:
    {data}

    # For any of these new columns, if the value is not found, you should just leave it blank.
    # Give a clean answer. Don't use the catergories names, just the values separeted by semicolon.
""")

# chain.batch(inputs=data)
categories = [(prompt | llm).invoke(input=d).content for d in data]

In [ ]:
jobs = pd.read_csv("data/jobs.csv")

In [ ]:
jobs["Model"] = jobs["Location"].apply(lambda l: l[l.find("(") + 1 : l.find(")")] if "(" in l else "")
jobs["Location"] = jobs["Location"].apply(lambda l: l[:l.find("(") - 1] if "(" in l else l)
jobs["Status"] = jobs["Last Status"].apply(lambda s: s.split(" ")[-3].lower())
jobs["Updated"] = jobs["Last Status"].apply(lambda s: s.split(" ")[-2])

In [ ]:
jobs["Updated"] = jobs["Updated"].apply(lambda t: "0yr" if not t.endswith("yr") else t)
jobs["When"] = jobs["Updated"].apply(lambda y: (dt.datetime.today() - dt.timedelta(weeks=52 * int(y[0]))).year)

In [ ]:
jobs.drop(columns=["Last Status"]).to_excel("data/jobs.xlsx", index=False)